In [ ]:
import time
import random
from multiprocessing import Pool
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

def generate_email(i):
    return f"ahmedyaqoobbusiness+{i}@gmail.com"

def get_api_key(index):
    chrome_options = Options()
    chrome_options.add_argument("--headless")  # remove if you want to see the browser
    driver = webdriver.Chrome(options=chrome_options)

    try:
        driver.get("https://www.alphavantage.co/support/#api-key")
        time.sleep(2)

        # Fill the form
        driver.find_element(By.ID, "occupation-text").send_keys("Student")
        driver.find_element(By.ID, "organization-text").send_keys("Fast National University")
        driver.find_element(By.ID, "email-text").send_keys(generate_email(index))
        time.sleep(1)

        # Enable and click the button via JS (the button is initially disabled)
        driver.execute_script("document.getElementById('submit-btn').disabled = false;")
        driver.find_element(By.ID, "submit-btn").click()
        time.sleep(5)

        # Get API key from response
        key_element = driver.find_element(By.ID, "talk").text
        api_key = key_element.split("access key is:")[-1].strip().split(".")[0]
        print(f"[{index}] Key: {api_key}")
        return api_key
    except Exception as e:
        print(f"[{index}] Error: {e}")
        return None
    finally:
        driver.quit()

def run_parallel(n=140):
    with Pool(processes=10) as pool:
        results = pool.map(get_api_key, range(n))

    # Save to file
    with open("alpha_vantage_api_keys.txt", "w") as f:
        for key in results:
            if key:
                f.write(key + "\n")

if __name__ == "__main__":
    run_parallel()
